# Проект: предсказание продаж интернет-магазина

## Описание
Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи.   
Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

## Цель
Предсказать вероятность покупки в течение 90 дней

## Задачи
- Изучить данные
- Разработать полезные признаки
- Создать модель для классификации пользователей
- Улучшить модель и максимизировать метрику roc_auc (минимум до 0.7)
- Выполнить тестирование
- Создать: 
    - тетрадь jupyter notebook с описанием, подготовкой признаков, обучением модели и тестированием
    - описание проекта и инструкция по использованию в файле README.md
    - список зависимостей в файле requirements.txt

## Описание данных
`apparel-purchases.csv` (история покупок):  

Данные о покупках клиентов по дням и по товарам. 
В каждой записи покупка определенного товара, его цена, количество штук.  
В таблице есть списки идентификаторов, к каким категориям относится товар.   
Часто это вложенные категории (например автотовары-аксессуары-освежители),   
но также может включать в начале списка маркер распродажи или маркер женщинам/мужчинам.  

Нумерация категорий сквозная для всех уровней, то есть 44 на второй позиции списка или на третьей – это одна и та же категория.   
Иногда дерево категорий обновляется, поэтому могут меняться вложенности, например ['4', '28', '44', '1594'] или ['4', '44', '1594'].   
Как обработать такие случаи – можете предлагать свои варианты решения.  

- `client_id` идентификатор пользователя  
- `quantity` количество товаров в заказе  
- `price` цена товара  
- `category_ids` вложенные категории, к которым отнсится товар  
- `date` дата покупки  
- `message_id` идентификатор сообщения из рассылки  

`apparel-messages.csv` (история рекламных рассылок):  

Рассылки, которые были отправлены клиентам из таблицы покупок.

- `bulk_campaign_id` идентификатор рекламной кампании  
- `client_id` идентификатор пользователя  
- `message_id` идентификатор сообщений  
- `event` тип действия  
- `channel` канал рассылки  
- `date` дата рассылки  
- `created_at` точное время создания сообщения  

`apparel-target_binary.csv` (совершит ли клиент покупку в течение следующих 90 дней):  
- `client_id` идентификатор пользователя  
- `target` клиент совершил покупку в целевом периоде (целевой признак)  

`full_campaign_daily_event.csv` (агрегация общей базы рассылок по дням и типам событий):  

Общая база рассылок огромна, поэтому собрали для вас агрегированную по дням  
статистику по рассылкам. Если будете создавать на основе этой статистики дополнительные  
признаки, обратите внимание, что нельзя суммировать по колонкам nunique, потому что это  
уникальные клиенты в пределах дня, у вас нет данных, повторяются ли они в другие дни  

- `date` дата
- `bulk_campaign_id` идентификатор рассылки
- `count_event`* общее количество каждого события event
- `nunique_event`* количество уникальных client_id в каждом событии  
*в именах колонок есть все типы событий event

`full_campaign_daily_event_channel.csv` (агрегация по дням с учетом событий и каналов рассылки):  
- `date` дата
- `bulk_campaign_id` идентификатор рассылки
- `count_event`*_channel* общее количество каждого события по каналам
- `nunique_event`*_channel* количество уникальных client_id по событиям и каналам   
_в именах колонок есть все типы событий event и каналов рассылки channel

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# оформление
pd.set_option('display.max_columns', None)

## Загрузка и предобработка данных

In [2]:
# грузим данные
df_purchases = pd.read_csv(r'data\apparel-purchases.csv')
df_messages = pd.read_csv(r'data\apparel-messages.csv')
df_target = pd.read_csv(r'data\apparel-target_binary.csv')
df_campaign = pd.read_csv(r'data\full_campaign_daily_event.csv')
df_campaign_channel = pd.read_csv(r'data\full_campaign_daily_event_channel.csv')

# делаем словарь
data_dict = {
    'df_purchases': df_purchases,
    'df_messages': df_messages,
    'df_target': df_target,
    'df_campaign': df_campaign,
    'df_campaign_channel': df_campaign_channel
}

for name, df in data_dict.items():
    print('-'*120, name)
    df.info()
    display(df.describe(include='all'))
    display(df.head())

------------------------------------------------------------------------------------------------------------------------ df_purchases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202208 entries, 0 to 202207
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   client_id     202208 non-null  int64  
 1   quantity      202208 non-null  int64  
 2   price         202208 non-null  float64
 3   category_ids  202208 non-null  object 
 4   date          202208 non-null  object 
 5   message_id    202208 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.3+ MB


,client_id,quantity,price,category_ids,date,message_id
count,2.022080e+05,202208.000000,202208.000000,202208,202208,202208
unique,NaN,NaN,NaN,933,642,50204
top,NaN,NaN,NaN,"['4', '28', '57', '431']",2022-11-11,1515915625489095763-6251-6311b13a4cf78
freq,NaN,NaN,NaN,8626,5270,365
mean,1.515916e+18,1.006483,1193.301516,NaN,NaN,NaN
std,1.459458e+08,0.184384,1342.252664,NaN,NaN,NaN
min,1.515916e+18,1.000000,1.000000,NaN,NaN,NaN
25%,1.515916e+18,1.000000,352.000000,NaN,NaN,NaN
50%,1.515916e+18,1.000000,987.000000,NaN,NaN,NaN
75%,1.515916e+18,1.000000,1699.000000,NaN,NaN,NaN


,client_id,quantity,price,category_ids,date,message_id
0,1515915625468169594,1,1999.0,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
1,1515915625468169594,1,2499.0,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
2,1515915625471138230,1,6499.0,"['4', '28', '57', '431']",2022-05-16,1515915625471138230-4437-6282242f27843
3,1515915625471138230,1,4999.0,"['4', '28', '244', '432']",2022-05-16,1515915625471138230-4437-6282242f27843
4,1515915625471138230,1,4999.0,"['4', '28', '49', '413']",2022-05-16,1515915625471138230-4437-6282242f27843


------------------------------------------------------------------------------------------------------------------------ df_messages
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12739798 entries, 0 to 12739797
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   bulk_campaign_id  int64 
 1   client_id         int64 
 2   message_id        object
 3   event             object
 4   channel           object
 5   date              object
 6   created_at        object
dtypes: int64(2), object(5)
memory usage: 680.4+ MB


,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
count,1.273980e+07,1.273980e+07,12739798,12739798,12739798,12739798,12739798
unique,NaN,NaN,9061667,11,2,638,4103539
top,NaN,NaN,1515915625489095763-6251-6311b13a4cf78,send,mobile_push,2023-06-10,2023-12-29 15:20:53
freq,NaN,NaN,1454,9058196,7512156,89661,621
mean,1.160459e+04,1.515916e+18,NaN,NaN,NaN,NaN,NaN
std,3.259211e+03,1.329704e+08,NaN,NaN,NaN,NaN,NaN
min,5.480000e+02,1.515916e+18,NaN,NaN,NaN,NaN,NaN
25%,8.746000e+03,1.515916e+18,NaN,NaN,NaN,NaN,NaN
50%,1.351600e+04,1.515916e+18,NaN,NaN,NaN,NaN,NaN
75%,1.415800e+04,1.515916e+18,NaN,NaN,NaN,NaN,NaN


...

,date,bulk_campaign_id,count_click,count_complain,count_hard_bounce,count_open,count_purchase,count_send,count_soft_bounce,count_subscribe,count_unsubscribe,nunique_click,nunique_complain,nunique_hard_bounce,nunique_open,nunique_purchase,nunique_send,nunique_soft_bounce,nunique_subscribe,nunique_unsubscribe,count_hbq_spam,nunique_hbq_spam,count_close,nunique_close
count,131072,131072.000000,131072.000000,131072.000000,131072.000000,1.310720e+05,131072.000000,1.310720e+05,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,1.310720e+05,131072.000000,1.310720e+05,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000
unique,727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2023-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,8416.743378,90.982971,0.932655,78.473434,3.771091e+03,0.577927,1.163414e+04,27.807312,0.140518,6.362679,74.276016,0.921326,77.398689,3.683000e+03,0.465103,1.153716e+04,27.573799,0.134125,5.960602,0.810364,0.809799,0.000008,0.000008
std,NaN,4877.369306,1275.503564,30.198326,1961.317826,6.516067e+04,9.107040,1.757095e+05,736.944714,2.072777,79.172069,1004.271405,29.715170,1913.395511,6.258647e+04,7.126368,1.727005e+05,734.050700,1.976439,73.284148,183.298579,183.298245,0.002762,0.002762
min,NaN,548.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,4116.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,7477.000000,0.000000,0.000000,0.000000,6.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,13732.000000,2.000000,0.000000,0.000000,3.000000e+01,0.000000,0.000000e+00,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000e+01,0.000000,0.000000e+00,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000


,date,bulk_campaign_id,count_click,count_complain,count_hard_bounce,count_open,count_purchase,count_send,count_soft_bounce,count_subscribe,count_unsubscribe,nunique_click,nunique_complain,nunique_hard_bounce,nunique_open,nunique_purchase,nunique_send,nunique_soft_bounce,nunique_subscribe,nunique_unsubscribe,count_hbq_spam,nunique_hbq_spam,count_close,nunique_close
0,2022-05-19,563,0,0,0,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


------------------------------------------------------------------------------------------------------------------------ df_campaign_channel
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131072 entries, 0 to 131071
Data columns (total 36 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   date                             131072 non-null  object
 1   bulk_campaign_id                 131072 non-null  int64 
 2   count_click_email                131072 non-null  int64 
 3   count_click_mobile_push          131072 non-null  int64 
 4   count_open_email                 131072 non-null  int64 
 5   count_open_mobile_push           131072 non-null  int64 
 6   count_purchase_email             131072 non-null  int64 
 7   count_purchase_mobile_push       131072 non-null  int64 
 8   count_soft_bounce_email          131072 non-null  int64 
 9   count_subscribe_email            131072 non-null  int64 
 10 

,date,bulk_campaign_id,count_click_email,count_click_mobile_push,count_open_email,count_open_mobile_push,count_purchase_email,count_purchase_mobile_push,count_soft_bounce_email,count_subscribe_email,count_unsubscribe_email,nunique_click_email,nunique_click_mobile_push,nunique_open_email,nunique_open_mobile_push,nunique_purchase_email,nunique_purchase_mobile_push,nunique_soft_bounce_email,nunique_subscribe_email,nunique_unsubscribe_email,count_hard_bounce_mobile_push,count_send_mobile_push,nunique_hard_bounce_mobile_push,nunique_send_mobile_push,count_hard_bounce_email,count_hbq_spam_email,count_send_email,nunique_hard_bounce_email,nunique_hbq_spam_email,nunique_send_email,count_soft_bounce_mobile_push,nunique_soft_bounce_mobile_push,count_complain_email,nunique_complain_email,count_close_mobile_push,nunique_close_mobile_push
count,131072,131072.000000,131072.000000,131072.000000,1.310720e+05,1.310720e+05,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,1.310720e+05,1.310720e+05,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,1.310720e+05,131072.000000,1.310720e+05,131072.000000,131072.000000,1.310720e+05,131072.000000,131072.000000,1.310720e+05,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000,131072.000000
unique,727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2023-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,8416.743378,41.582169,49.400803,4.237060e+02,3.347385e+03,0.357483,0.220444,24.474823,0.140518,6.362679,31.396263,42.879753,4.116615e+02,3.271339e+03,0.287712,0.177391,24.262146,0.134125,5.960602,59.483444,7.444562e+03,58.863007,7.350267e+03,18.989990,0.810364,4.189581e+03,18.535683,0.809799,4.186898e+03,3.332489,3.311653,0.932655,0.921326,0.000008,0.000008
std,NaN,4877.369306,745.484035,1036.952898,9.753384e+03,6.444859e+04,8.287483,3.796500,727.069387,2.072777,79.172069,562.883309,833.316257,9.519713e+03,6.188001e+04,6.484979,2.971908,724.270910,1.976439,73.284148,1371.955350,1.393509e+05,1357.271261,1.355799e+05,1402.414107,183.298579,1.073198e+05,1349.473695,183.298245,1.072618e+05,120.916269,120.094858,30.198326,29.715170,0.002762,0.002762
min,NaN,548.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,4116.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,7477.000000,0.000000,0.000000,5.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,13732.000000,1.000000,0.000000,2.300000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,2.300000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,date,bulk_campaign_id,count_click_email,count_click_mobile_push,count_open_email,count_open_mobile_push,count_purchase_email,count_purchase_mobile_push,count_soft_bounce_email,count_subscribe_email,count_unsubscribe_email,nunique_click_email,nunique_click_mobile_push,nunique_open_email,nunique_open_mobile_push,nunique_purchase_email,nunique_purchase_mobile_push,nunique_soft_bounce_email,nunique_subscribe_email,nunique_unsubscribe_email,count_hard_bounce_mobile_push,count_send_mobile_push,nunique_hard_bounce_mobile_push,nunique_send_mobile_push,count_hard_bounce_email,count_hbq_spam_email,count_send_email,nunique_hard_bounce_email,nunique_hbq_spam_email,nunique_send_email,count_soft_bounce_mobile_push,nunique_soft_bounce_mobile_push,count_complain_email,nunique_complain_email,count_close_mobile_push,nunique_close_mobile_push
0,2022-05-19,563,0,0,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2022-05-19,577,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2022-05-19,622,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2022-05-19,634,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2022-05-19,676,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Выводы:
- Purchases
    - id сделать строкой (и проверить нет ли случаев, когда разная длина строки, вдруг нули обрубились в начале)
    - date перевести в формат даты
- Messages
    - id сделать строкой (и проверить нет ли случаев, когда разная длина строки, вдруг нули обрубились в начале)
    - столбцы дат перевести в формат даты
- Target
    - id сделать строкой (и проверить нет ли случаев, когда разная длина строки, вдруг нули обрубились в начале)
- Campaign Events
    - id сделать строкой (и проверить нет ли случаев, когда разная длина строки, вдруг нули обрубились в начале)
    - date перевести в формат даты
- Campaign Events by Channel
    - id сделать строкой (и проверить нет ли случаев, когда разная длина строки, вдруг нули обрубились в начале)
    - date перевести в формат даты


In [3]:
# проверка на неяывные дубликаты
for name, df in data_dict.items():
    print(f'{name}:', end='\n')

    for col in df.columns:
        if col.endswith('_id'):
            df[col] = df[col].astype(str)
            print(f'{df[col].str.len().value_counts().to_string()}', end='\n\n')
        if col in ['date','created_at']:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    for col in df.select_dtypes('object').columns:
        if not col.endswith('_id') and not col.endswith('_ids'):
            print(f'Уникальные значения в столбце {col}: \n{sorted(df[col].unique())}', end='\n\n')


df_purchases:
client_id
19    202208

message_id
39    116465
38     85721
37        22

df_messages:
bulk_campaign_id
5    8768121
4    3971550
3        127

client_id
19    12739798

message_id
39    8768134
38    3971537
37        127

Уникальные значения в столбце event: 
['click', 'close', 'complain', 'hard_bounce', 'hbq_spam', 'open', 'purchase', 'send', 'soft_bounce', 'subscribe', 'unsubscribe']

Уникальные значения в столбце channel: 
['email', 'mobile_push']

df_target:
client_id
19    49849

df_campaign:
bulk_campaign_id
4    73859
5    53775
3     3438

df_campaign_channel:
bulk_campaign_id
4    73859
5    53775
3     3438



Кажется, что идентификатор рекламной кампании не всегда должен быть одного формата,   
т.к. в теории им могли просто порядковые номера исторической последовательности присваивать,  
а идентификатор сообщения это комбинация айди клиента, айди кампании и строкове значение,   
поэтому вполне логично, что если значения айди маркетинговой кампании на один меньше,   
то и строки с айди сообещния будут с разницей в такое же количество символов.

В идеале этот момент уточнить у источника данных, а в рамках этого исследования мы это опустим и пойдем дальше. 

Сейчас займемся поиском пропусков и дубликатов:

In [4]:
# проверим на пропуски
for name, df in data_dict.items():
    if df.isna().any().any() or df.isnull().any().any():
        print('-'*120, name)
        display(df[df.isna().any(axis=1) | df.isnull().any(axis=1)].head())
    else:
        print(f'{name}: пропусков нет')

df_purchases: пропусков нет
df_messages: пропусков нет
df_target: пропусков нет
df_campaign: пропусков нет
df_campaign_channel: пропусков нет


In [5]:
# проверим на дубликаты
for name, df in data_dict.items():
    if df.duplicated().any():
        print(f'{name}: {df.duplicated().sum()} дубликатов ({df.duplicated().sum() / len(df) * 100:.2f}% от общего количества строк)')
        display(df[df.duplicated(keep=False)].head(2))
    else:
        print(f'{name}: дубликатов нет')

df_purchases: 73020 дубликатов (36.11% от общего количества строк)


,client_id,quantity,price,category_ids,date,message_id
11,1515915625491869271,2,599.0,"['4', '27', '350', '1392']",2022-05-16,1515915625491869271-2090-61a72488d6a0f
12,1515915625491869271,2,599.0,"['4', '27', '350', '1392']",2022-05-16,1515915625491869271-2090-61a72488d6a0f


df_messages: 48610 дубликатов (0.38% от общего количества строк)


,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
964231,5723,1515915625554535987,1515915625554535987-5723-62e2af08e00da,click,mobile_push,2022-07-28,2022-07-28 15:58:56
964232,5723,1515915625554535987,1515915625554535987-5723-62e2af08e00da,click,mobile_push,2022-07-28,2022-07-28 15:58:56


df_target: дубликатов нет
df_campaign: дубликатов нет
df_campaign_channel: дубликатов нет


Если предположить, что постфикс айди сообщения *1515915625491869271-2090-**61a72488d6a0f*** разный при каждой следующей покупке,  
то это действительно дубликаты, которые следует удалить (если есть возможность сперва уточнить у источника данных, т.к. дублей треть файла покупок). 

В файле сообщений таких строк значит меньше, менее 1%, их можно смело удалить:

In [6]:
# удаляем дубликаты
df_purchases.drop_duplicates(inplace=True)
df_purchases.reset_index(drop=True, inplace=True)

df_messages.drop_duplicates(inplace=True)
df_messages.reset_index(drop=True, inplace=True)

# снова проверим на дубликаты
for name, df in data_dict.items():
    if df.duplicated().any():
        print(f'{name}: {df.duplicated().sum()} дубликатов ({df.duplicated().sum() / len(df) * 100:.2f}% от общего количества строк)')
        display(df[df.duplicated(keep=False)].head(2))
    else:
        print(f'{name}: дубликатов нет')

df_purchases: дубликатов нет
df_messages: дубликатов нет
df_target: дубликатов нет
df_campaign: дубликатов нет
df_campaign_channel: дубликатов нет


## Исследование данных


## Инжиниринг признаков


## Обучение модели
Разработаем модель, которая будет предсказывать вероятность покупки в течение 90 дней

## Тестирование модели
